In [35]:
import openai
import os
%load_ext dotenv
%dotenv
openai.api_key = os.getenv("OPENAI_API_KEY")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [36]:
text_to_parse = input("Enter utterance for parsing")
print(text_to_parse)

the upcming Sonday 13pm


In [37]:
spell_correction_prompt = """
Try correcting the spelling of these phrases about date and time:
Input: 'Teuesdey oif Nexx weak att 9 pm'
Output: 'Tuesday of Next week at 9 pm'

Input: 'Frday at nooon'
Output: 'Friday at noon'

Input: 'Monday 3 pm'
Output: 'Monday 3 pm'

Input: '{}'
Output: ?
""".format(text_to_parse)

print(spell_correction_prompt)


Try correcting the spelling of these phrases about date and time:
Input: 'Teuesdey oif Nexx weak att 9 pm'
Output: 'Tuesday of Next week at 9 pm'

Input: 'Frday at nooon'
Output: 'Friday at noon'

Input: 'Monday 3 pm'
Output: 'Monday 3 pm'

Input: 'the upcming Sonday 13pm'
Output: ?



In [38]:
spell_correction_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "system", "content": spell_correction_prompt}],
    max_tokens=50)


In [39]:
raw_spell_corrected_message = spell_correction_response.choices[0].message.content.strip()
print(raw_spell_corrected_message)

Output: 'The upcoming Sunday at 1 pm'


In [40]:
import re
spell_correction_matched = re.search("(?:Output:\s*)?'([^']+?)'", raw_spell_corrected_message)
if spell_correction_matched is not None and len(spell_correction_matched.groups()) == 1:
    spell_corrected_text_to_parse = spell_correction_matched.group(1)
else:
    spell_corrected_text_to_parse = text_to_parse
    print("Unrecognized return format")
print("Entering semantic parsing for {}...".format(spell_corrected_text_to_parse))

Entering semantic parsing for The upcoming Sunday at 1 pm...


In [41]:
from datetime import date

semantic_parsing_prompt = """
Follow my examples and give me an answer.

Assuming reference date = '2023-02-24'
'Tomorrow at 11am' -> '2023-02-25 11:00'
'This coming Tuesday at 9pm' -> '2023-02-28 21:00'
'Next Tuesday at 9pm' -> '2023-02-28 21:00'
'Teuesdey oif Nexx weak att 9 pm' -> '2023-02-28 21:00'

Reference date = '{}'
'{}' -> ?
""".format(date.today(), spell_corrected_text_to_parse)

print(semantic_parsing_prompt)


Follow my examples and give me an answer.

Assuming reference date = '2023-02-24'
'Tomorrow at 11am' -> '2023-02-25 11:00'
'This coming Tuesday at 9pm' -> '2023-02-28 21:00'
'Next Tuesday at 9pm' -> '2023-02-28 21:00'
'Teuesdey oif Nexx weak att 9 pm' -> '2023-02-28 21:00'

Reference date = '2023-03-07'
'The upcoming Sunday at 1 pm' -> ?



In [42]:
semantic_parsing_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[{"role": "system", "content": semantic_parsing_prompt}],
    max_tokens=50)


In [43]:
raw_semantic_parsing_message = semantic_parsing_response.choices[0].message.content.strip()
print(raw_semantic_parsing_message)

'2023-03-12 13:00'


In [44]:
import re
semantic_parsing_matched = re.search("'(\d\d\d\d-\d\d-\d\d \d\d:\d\d)'", raw_semantic_parsing_message)
if semantic_parsing_matched is not None and len(semantic_parsing_matched.groups()) == 1:
    print(semantic_parsing_matched.group(1))
else:
    print("Unrecognized return format")

2023-03-12 13:00
